# Lab 11: Grid Localization using Bayes Filter (Virtual Robot)

### <span style="color:rgb(0,150,0)">It is recommended that you close any heavy-duty applications running on your system while working on this lab.</span>

<hr>


In [1]:
%load_ext autoreload
%autoreload 2

import traceback
from notebook_utils import *
from Traj import *
import asyncio
from localization_extras import Localization

# Setup Logger
LOG = get_logger('demo_notebook.log')

# Init GUI and Commander
gui = GET_GUI()
cmdr = gui.launcher.commander

gui.show()

# Start the simulator
START_SIM()

# Start the plotter
START_PLOTTER()

2025-04-22 14:24:47,852 | INFO     |: Logger demo_notebook.log initialized.


TwoByTwoLayout(children=(Label(value='Simulator', layout=Layout(grid_area='top-left', width='80px')), HBox(chi…

Loading Flatland...
Initializing pygame framework...


2025-04-22 14:24:48.429 Python[44063:13002745] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.


In [2]:
# Initialize Robot to communicate with the virtual robot and plotter
robot = VirtualRobot(cmdr)

# Initialize mapper
# Requires a VirtualRobot object as a parameter
mapper = Mapper(robot)

# Initialize your BaseLocalization object
# Requires a VirtualRobot object and a Mapper object as parameters
loc = Localization(robot, mapper)

## Plot Map
cmdr.plot_map()

2025-04-22 14:24:52,525 | INFO     |:  | Number of observations per grid cell: 18
2025-04-22 14:24:52,526 | INFO     |:  | Precaching Views...


/Users/akin/Desktop/Desktop/dev/fastrobots/sim/localization.py:150: RuntimeWarning: All-NaN slice encountered
  return np.nanmin(distance_intersections_tt), intersections_tt[np.nanargmin(distance_intersections_tt)]


2025-04-22 14:24:53,649 | INFO     |:  | Precaching Time: 1.123 secs
2025-04-22 14:24:53,650 | INFO     |: Initializing beliefs with a Uniform Distribution
2025-04-22 14:24:53,650 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107


# Run the Bayes Filter
The cells below utilizes the member functions of class **Localization** (defined in [localization_extras.py](../localization_extras.py)) in each iteration of the Bayes filter algorithm to localize the robot in the grid map. <br>

In [3]:
# Reset Robot and Plots
robot.reset()
cmdr.reset_plotter()

# Init Uniform Belief
loc.init_grid_beliefs()

# Get Observation Data by executing a 360 degree rotation motion
loc.get_observation_data()

# Run Update Step
loc.update_step()
loc.print_update_stats(plot_data=True)

# Plot Odom and GT
current_odom, current_gt = robot.get_pose()
cmdr.plot_gt(current_gt[0], current_gt[1])
cmdr.plot_odom(current_odom[0], current_odom[1])

2025-04-22 14:24:56,003 | INFO     |: Initializing beliefs with a Uniform Distribution
2025-04-22 14:24:56,003 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107
2025-04-22 14:24:58,952 | INFO     |: Update Step
2025-04-22 14:24:58,954 | INFO     |:      | Update Time: 0.001 secs
2025-04-22 14:24:58,954 | INFO     |: ---------- UPDATE STATS -----------
2025-04-22 14:24:58,966 | INFO     |: GT index      : (6, 4, 9)
2025-04-22 14:24:58,966 | INFO     |: Bel index     : (np.int64(5), np.int64(4), np.int64(9)) with prob = 0.9985028
2025-04-22 14:24:58,967 | INFO     |: Bel_bar prob at index = 0.00051440329218107
2025-04-22 14:24:58,967 | INFO     |: GT            : (0.000, 0.000, 360.000)
2025-04-22 14:24:58,967 | INFO     |: Belief        : (0.000, 0.000, 10.000)
2025-04-22 14:24:58,968 | INFO     |: POS ERROR     : (-0.000, -0.000, 350.000)
2025-04-22 14:24:58,968 | INFO     |: ---------- UPDATE STATS -----------


In [4]:
# Initialize the Trajectory object
traj = Trajectory(loc)

# Run through each motion steps
for t in range(0, traj.total_time_steps):
    print("\n\n-----------------", t, "-----------------")
    
    prev_odom, current_odom, prev_gt, current_gt = traj.execute_time_step(t)
        
    # Prediction Step
    loc.prediction_step(current_odom, prev_odom)
    loc.print_prediction_stats(plot_data=True)
    
    # Get Observation Data by executing a 360 degree rotation motion
    loc.get_observation_data()
    
    # Update Step
    loc.update_step()
    loc.print_update_stats(plot_data=True)

# Uncomment the below line to wait for keyboard input between each iteration.
#   input("Press Enter to Continue")
        
    print("-------------------------------------")



----------------- 0 -----------------


Traceback (most recent call last):
  File "/Users/akin/Desktop/Desktop/dev/fastrobots/sim/src/plotter.py", line 263, in keyPressEvent
    if event.key() == Qt.Key_Escape:
                      ^^^^^^^^^^^^^
AttributeError: type object 'Qt' has no attribute 'Key_Escape'


2025-04-22 14:25:07,836 | INFO     |: Prediction Step
2025-04-22 14:25:07,869 | INFO     |:  | Prediction Time: 0.032 secs
2025-04-22 14:25:07,870 | INFO     |: ---------- PREDICTION STATS -----------
2025-04-22 14:25:07,887 | INFO     |: GT index         : (6, 3, 6)
2025-04-22 14:25:07,887 | INFO     |: Prior Bel index  : (np.int64(6), np.int64(2), np.int64(9)) with prob = 0.0891802
2025-04-22 14:25:07,888 | INFO     |: POS ERROR        : (-0.019, 0.520, -50.012)
2025-04-22 14:25:07,888 | INFO     |: ---------- PREDICTION STATS -----------
2025-04-22 14:25:10,835 | INFO     |: Update Step
2025-04-22 14:25:10,837 | INFO     |:      | Update Time: 0.001 secs
2025-04-22 14:25:10,837 | INFO     |: ---------- UPDATE STATS -----------
2025-04-22 14:25:10,847 | INFO     |: GT index      : (6, 3, 6)
2025-04-22 14:25:10,848 | INFO     |: Bel index     : (np.int64(6), np.int64(3), np.int64(7)) with prob = 0.9586657
2025-04-22 14:25:10,848 | INFO     |: Bel_bar prob at index = 0.0001229920340186

Traceback (most recent call last):
  File "/Users/akin/Desktop/Desktop/dev/fastrobots/sim/src/plotter.py", line 263, in keyPressEvent
    if event.key() == Qt.Key_Escape:
                      ^^^^^^^^^^^^^
AttributeError: type object 'Qt' has no attribute 'Key_Escape'
Traceback (most recent call last):
  File "/Users/akin/Desktop/Desktop/dev/fastrobots/sim/src/plotter.py", line 263, in keyPressEvent
    if event.key() == Qt.Key_Escape:
                      ^^^^^^^^^^^^^
AttributeError: type object 'Qt' has no attribute 'Key_Escape'
Traceback (most recent call last):
  File "/Users/akin/Desktop/Desktop/dev/fastrobots/sim/src/plotter.py", line 263, in keyPressEvent
    if event.key() == Qt.Key_Escape:
                      ^^^^^^^^^^^^^
AttributeError: type object 'Qt' has no attribute 'Key_Escape'
Traceback (most recent call last):
  File "/Users/akin/Desktop/Desktop/dev/fastrobots/sim/src/plotter.py", line 263, in keyPressEvent
    if event.key() == Qt.Key_Escape:
                   